<a href="https://colab.research.google.com/github/jennif8r/notebookLM/blob/main/docling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install docling

In [3]:
pip install whisper langchain chromadb cohere pypdfium2 langchain-community

#Tabela e texto


##Passo 1: Configuração e Inicialização


In [30]:
import json
import yaml
import logging
from time import time
import os
import uuid
import chromadb
from chromadb.config import Settings
from cohere import Client
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import pandas as pd
from docling.backend.pypdfium2_backend import PyPdfiumDocumentBackend
from docling.datamodel.base_models import InputFormat
from docling.document_converter import (
    DocumentConverter,
    PdfFormatOption,
    WordFormatOption,
)
from docling.pipeline.simple_pipeline import SimplePipeline
from docling.pipeline.standard_pdf_pipeline import StandardPdfPipeline

# Inicializar cliente Cohere
cohere_api_key = "senha"
cohere_client = Client(api_key=cohere_api_key)

# Configurar cliente ChromaDB
client = chromadb.Client()
collection_dimensions = 768  # Dimensão vetorial


###Passo 2: Função para Extrair Conteúdo e Tabelas


###[desatualizado]

In [79]:
def extract_content_and_tables_from_file(file_path):
    converter = DocumentConverter()

    # Converte o arquivo
    result = converter.convert(file_path)
    document = result.document

    # Extrair conteúdo textual
    text_content = document.export_to_text()

    # Extrair tabelas
    tables = []
    for table_ix, table in enumerate(document.tables):
        table_df = table.export_to_dataframe()
        tables.append({
            'index': table_ix + 1,
            'data': table_df
        })

    return text_content, tables


Teste este código para verificar se o conteúdo e as tabelas estão sendo extraídos corretamente:


In [81]:
file_path = "/content/example_01.html"
text_content, tables = extract_content_and_tables_from_file(file_path)
print("Conteúdo extraído:\n", text_content)
for table in tables:
    print(f"Tabela {table['index']}:\n", table['data'])


Conteúdo extraído:
 Introduction

This is the first paragraph of the introduction.

Background

Some background information here.

<missing-text>

 First item in unordered list

 Second item in unordered list

 First item in ordered list

 Second item in ordered list


###[Atualização]

Arquivo como TXT e CSV que não são processados pela biblioteca irei converter para algum tipo de arquivo que a biblioteca aceita.

Foi testado com os seguintes arquivos e funcionou:
- mp3, m4a e ogg
-docx, md, xlsx, csv, pptx, html, txt, pdf,png

In [43]:
pip install pandas openpyxl fpdf


  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=c31e2e22d6ec0ca405581c937566b59c0ebf794390777ce7821c774d05d28e64
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [1]:
pip install openai-whisper


In [116]:
import json
from pathlib import Path
import pandas as pd
from fpdf import FPDF
from docling.document_converter import DocumentConverter
import whisper


# Configurações globais de arquivos suportados
WHISPER_SUPPORTED_AUDIO = ['.mp3', '.ogg', '.wav', '.flac', '.m4a']
DOCLING_SUPPORTED_FILES = ['.md', '.html', '.docx', '.pptx', '.png', '.pdf','.xlsx']
ADDITIONAL_CONVERTED_FILES = ['.txt', '.csv']


def transcribe_audio(file_path):
    """Transcreve o áudio usando Whisper."""
    try:
        model = whisper.load_model("base")
        result = model.transcribe(str(file_path), language="pt")
        return result['text']
    except Exception as e:
        return f"Erro ao transcrever o áudio: {e}"


def convert_to_supported_format(file_path):
    """
    Converte arquivos TXT e CSV para um formato suportado pela biblioteca docling.
    Retorna o caminho do arquivo convertido.
    """
    file_path = Path(file_path)
    if file_path.suffix.lower() == '.txt':
        pdf_path = file_path.with_suffix('.pdf')
        txt_to_pdf(file_path, pdf_path)
        return pdf_path
    elif file_path.suffix.lower() == '.csv':
        xlsx_path = file_path.with_suffix('.xlsx')
        csv_to_xlsx(file_path, xlsx_path)
        return xlsx_path
    return file_path  # Retorna o arquivo original se não precisar de conversão


def txt_to_pdf(txt_file, pdf_file):
    """Converte um arquivo TXT para PDF usando FPDF."""
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    with open(txt_file, 'r', encoding='utf-8') as file:
        for line in file:
            pdf.cell(0, 10, line.strip(), ln=True)
    pdf.output(pdf_file)
    print(f"Arquivo TXT convertido para PDF: {pdf_file}")


def csv_to_xlsx(csv_file, xlsx_file):
    """Converte um arquivo CSV para XLSX usando Pandas."""
    df = pd.read_csv(csv_file)
    df.to_excel(xlsx_file, index=False, engine='openpyxl')
    print(f"Arquivo CSV convertido para XLSX: {xlsx_file}")


def extract_content_and_tables(file_path):
    """
    Extrai conteúdo textual e tabelas de um arquivo usando a biblioteca docling.
    """
    try:
        supported_file = convert_to_supported_format(file_path)
        converter = DocumentConverter()
        result = converter.convert(supported_file)
        document = result.document

        text_content = document.export_to_text()
        tables = [
            {'index': idx + 1, 'data': table.export_to_dataframe()}
            for idx, table in enumerate(document.tables)
        ]
        return text_content, tables
    except Exception as e:
        return f"Erro ao processar o documento: {e}", []


def is_supported_audio(file_path):
    """Verifica se o arquivo é um formato de áudio suportado pelo Whisper."""
    return file_path.suffix.lower() in WHISPER_SUPPORTED_AUDIO


def is_supported_document(file_path):
    """Verifica se o arquivo é um formato de documento suportado pelo Docling."""
    return file_path.suffix.lower() in DOCLING_SUPPORTED_FILES + ADDITIONAL_CONVERTED_FILES


def process_file(file_path):
    """
    Processa o arquivo com base em seu tipo:
    - Se for áudio, transcreve.
    - Se for documento, extrai texto e tabelas.
    - Caso contrário, informa que o tipo de arquivo não é suportado.
    """
    file_path = Path(file_path)

    if is_supported_audio(file_path):
        print(f"Arquivo de áudio detectado: {file_path.name}")
        print(f"Formatos suportados: {', '.join(WHISPER_SUPPORTED_AUDIO)}")
        return transcribe_audio(file_path), None

    elif is_supported_document(file_path):
        print(f"Arquivo de documento detectado: {file_path.name}")
        print(f"Formatos suportados: {', '.join(DOCLING_SUPPORTED_FILES + ADDITIONAL_CONVERTED_FILES)}")
        return extract_content_and_tables(file_path)

    else:
        print(f"Tipo de arquivo não suportado: {file_path.suffix}")
        print("Por favor, tente outro arquivo.")
        print(f"Formatos suportados pelo Whisper: {', '.join(WHISPER_SUPPORTED_AUDIO)}")
        print(f"Formatos suportados pelo Docling: {', '.join(DOCLING_SUPPORTED_FILES + ADDITIONAL_CONVERTED_FILES)}")
        return None, None


def display_results(text_content, tables):
    """Exibe o conteúdo textual e as tabelas extraídas."""
    if text_content:
        print("Conteúdo textual extraído:")
        print("=" * 50)
        print(text_content if isinstance(text_content, str) else "Nenhum conteúdo textual encontrado.")
        print("=" * 50)
    else:
        print("Nenhum conteúdo textual extraído.")

    if tables:
        print(f"\nForam encontradas {len(tables)} tabela(s):")
        for table in tables:
            print(f"\nTabela {table['index']}:\n")
            print(table['data'])
    else:
        print("\nNenhuma tabela encontrada no arquivo.")


# Exemplo de uso
if __name__ == "__main__":
    file_path = "/content/2305.03393v1-pg9-img.png"  # Substitua pelo caminho do arquivo
    text_content, tables = process_file(file_path)
    display_results(text_content, tables)


Arquivo de documento detectado: 2305.03393v1-pg9-img.png
Formatos suportados: .md, .html, .docx, .pptx, .png, .pdf, .xlsx, .txt, .csv
Conteúdo textual extraído:
order to compute the TED score. Inference timing results for all experiments were obtained from the same machine On a single core with AMD EPYC 7763 CPU @2.45 GHz.

5.1 Hyper Parameter Optimization

We have chosen the PubIabNet data set to perform HPO, since it includes à highly diverse set of tables. Also we report TED scores separately for simple and complex tables (tables with cell spans) Results are presented in Table. It is evident that with OTSL, our model achieves the same TED score and slightly better mAP scores in comparison to HIML. However OISL yields a 2r speed up in the inference runtime over HIML.

<missing-text>

5.2 Quantitative Results

We picked the model parameter configuration that produced the best prediction quality (enc=6, dec=6, heads=8) with PubTabNet  alone; then independently trained and evaluated it 

##Passo 3: Divisão em Chunks
Dividimos o conteúdo extraído em chunks.


In [117]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter

def create_chunks_from_text_and_tables(text_content, tables, file_name, chunk_size=500):
    """
    Divide o conteúdo de texto e tabelas em chunks utilizando LangChain.
    """
    # Inicializa o divisor de texto com LangChain
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=100
    )

    # Divide o texto em chunks
    text_chunks = text_splitter.create_documents([text_content])
    chunk_id = 1

    # Armazena os chunks gerados
    combined_chunks = []

    # Processa os chunks de texto
    for chunk in text_chunks:
        combined_chunks.append({
            'text': chunk.page_content,
            'metadata': {
                'id': chunk_id,  # Gera um ID único
                'length': len(chunk.page_content),
                'type': 'text',
                'file_name': file_name  # Adiciona o nome do arquivo aos metadados
            }
        })
        chunk_id += 1

    # Processa as tabelas extraídas (se existirem)
    if tables:  # Verifica se tables não é None ou uma lista vazia
        for table in tables:
            combined_chunks.append({
                'text': table['data'].to_markdown(),
                'metadata': {
                    'id': chunk_id,  # Gera um ID único para tabelas
                    'length': len(table['data']),
                    'type': 'table',
                    'index': table['index'],
                    'file_name': file_name  # Adiciona o nome do arquivo aos metadados
                }
            })
            chunk_id += 1

    return combined_chunks


# Testando a função
if __name__ == "__main__":
    # Certifique-se de que 'file_path', 'text_content', e 'tables' estão disponíveis do processamento anterior
    try:
        # Obtém o nome do arquivo
        file_name = os.path.basename(file_path)  # Necessário após importar os dados do arquivo

        # Inicializa tables como lista vazia para arquivos de áudio
        if tables is None:
            tables = []

        # Cria os chunks
        chunks = create_chunks_from_text_and_tables(text_content, tables, file_name)

        # Exibe os resultados
        print(f"{len(chunks)} chunks foram gerados:")
        for chunk in chunks:
            print(f"ID: {chunk['metadata']['id']}, Tipo: {chunk['metadata']['type']}")
            print(f"Texto/Conteúdo:\n{chunk['text']}\n")
    except NameError as e:
        print(f"Erro: Variáveis não definidas no escopo atual. Certifique-se de que 'text_content', 'tables', e 'file_path' estejam definidos.")
        print(f"Detalhes do erro: {e}")
    except TypeError as e:
        print(f"Erro: Tipo de dado inesperado. Certifique-se de que 'tables' seja uma lista ou 'None'.")
        print(f"Detalhes do erro: {e}")


6 chunks foram gerados:
ID: 1, Tipo: text
Texto/Conteúdo:
order to compute the TED score. Inference timing results for all experiments were obtained from the same machine On a single core with AMD EPYC 7763 CPU @2.45 GHz.

5.1 Hyper Parameter Optimization

ID: 2, Tipo: text
Texto/Conteúdo:
5.1 Hyper Parameter Optimization

We have chosen the PubIabNet data set to perform HPO, since it includes à highly diverse set of tables. Also we report TED scores separately for simple and complex tables (tables with cell spans) Results are presented in Table. It is evident that with OTSL, our model achieves the same TED score and slightly better mAP scores in comparison to HIML. However OISL yields a 2r speed up in the inference runtime over HIML.

<missing-text>

5.2 Quantitative Results

ID: 3, Tipo: text
Texto/Conteúdo:
We picked the model parameter configuration that produced the best prediction quality (enc=6, dec=6, heads=8) with PubTabNet  alone; then independently trained and evaluated it o

Teste a criação dos chunks:

In [118]:
print("Chunks gerados:", len(chunks))

for idx, chunk in enumerate(chunks, start=1):  # Enumerar os chunks a partir de 1
    print(f"\n--- Chunk {idx} ---")
    print(f"Texto do chunk:\n{chunk['text']}")
    print(f"Metadados: {chunk['metadata']}")


Chunks gerados: 6

--- Chunk 1 ---
Texto do chunk:
order to compute the TED score. Inference timing results for all experiments were obtained from the same machine On a single core with AMD EPYC 7763 CPU @2.45 GHz.

5.1 Hyper Parameter Optimization
Metadados: {'id': 1, 'length': 197, 'type': 'text', 'file_name': '2305.03393v1-pg9-img.png'}

--- Chunk 2 ---
Texto do chunk:
5.1 Hyper Parameter Optimization

We have chosen the PubIabNet data set to perform HPO, since it includes à highly diverse set of tables. Also we report TED scores separately for simple and complex tables (tables with cell spans) Results are presented in Table. It is evident that with OTSL, our model achieves the same TED score and slightly better mAP scores in comparison to HIML. However OISL yields a 2r speed up in the inference runtime over HIML.

<missing-text>

5.2 Quantitative Results
Metadados: {'id': 2, 'length': 495, 'type': 'text', 'file_name': '2305.03393v1-pg9-img.png'}

--- Chunk 3 ---
Texto do chunk:
We 

##Passo 4: Gerar Embeddings
Utilizamos o Cohere para gerar embeddings.


In [119]:
def generate_embeddings(text_list):
    response = cohere_client.embed(model="embed-multilingual-v2.0", texts=text_list)
    return response.embeddings


Teste a geração dos embeddings:


In [120]:
texts = [chunk['text'] for chunk in chunks]
embeddings = generate_embeddings(texts)
print("Quantidade de embeddings gerados:", len(embeddings))
for i, embedding in enumerate(embeddings[:3]):  # Mostre os primeiros 3 embeddings
    print(f"Embedding {i + 1}: {embedding[:10]}...")  # Exibir os primeiros 10 valores


Quantidade de embeddings gerados: 6
Embedding 1: [0.49121094, -0.19360352, 0.042907715, 0.15539551, -0.0019226074, -0.10412598, 0.20336914, -0.28295898, 0.09185791, 0.34179688]...
Embedding 2: [0.37329102, -0.17492676, -0.2915039, 0.45532227, 0.37280273, -0.33520508, 0.27368164, -0.22949219, 0.12207031, 0.51220703]...
Embedding 3: [0.31982422, -0.13269043, -0.010658264, 0.42016602, 0.028045654, -0.07507324, 0.025512695, -0.29052734, 0.043273926, 0.42163086]...


Combinando os embedding com os metadados

In [121]:
# Combinar embeddings com seus metadados, incluindo o campo 'content'
def combine_embeddings_with_metadata(embeddings, chunks):
    combined_data = []
    for embedding, chunk in zip(embeddings, chunks):
        combined_data.append({
            'embedding': embedding,  # O embedding gerado
            'metadata': {
                **chunk['metadata'],  # Mantém os metadados originais
                'content': chunk['text']  # Adiciona o conteúdo do chunk
            }
        })
    return combined_data

# Combinar embeddings e metadados
combined_data = combine_embeddings_with_metadata(embeddings, chunks)

# Exibir os dados combinados para validação
print("Quantidade de dados combinados:", len(combined_data))
for idx, data in enumerate(combined_data[:12], start=1):  # Mostrar os primeiros 3 para validação
    print(f"\n--- Dado {idx} ---")
    print(f"Embedding (resumido): {data['embedding'][:10]}...")  # Mostrar apenas os primeiros 10 valores
    print(f"Metadados: {data['metadata']}")


Quantidade de dados combinados: 6

--- Dado 1 ---
Embedding (resumido): [0.49121094, -0.19360352, 0.042907715, 0.15539551, -0.0019226074, -0.10412598, 0.20336914, -0.28295898, 0.09185791, 0.34179688]...
Metadados: {'id': 1, 'length': 197, 'type': 'text', 'file_name': '2305.03393v1-pg9-img.png', 'content': 'order to compute the TED score. Inference timing results for all experiments were obtained from the same machine On a single core with AMD EPYC 7763 CPU @2.45 GHz.\n\n5.1 Hyper Parameter Optimization'}

--- Dado 2 ---
Embedding (resumido): [0.37329102, -0.17492676, -0.2915039, 0.45532227, 0.37280273, -0.33520508, 0.27368164, -0.22949219, 0.12207031, 0.51220703]...
Metadados: {'id': 2, 'length': 495, 'type': 'text', 'file_name': '2305.03393v1-pg9-img.png', 'content': '5.1 Hyper Parameter Optimization\n\nWe have chosen the PubIabNet data set to perform HPO, since it includes à highly diverse set of tables. Also we report TED scores separately for simple and complex tables (tables with 

##Passo 5: Inserir no ChromaDB
Adicionamos os embeddings e metadados ao ChromaDB.

In [66]:
# Criar ou obter a coleção no ChromaDB
collection_name = "novo"
collection = client.get_or_create_collection(
    name=collection_name,
    metadata={"dimensions": collection_dimensions}
)

# Inserir embeddings, metadados e IDs na coleção
for chunk, embedding in zip(chunks, embeddings):
    # Atualizar os metadados para incluir o campo 'content'
    metadata_with_content = {
        **chunk['metadata'],  # Mantém os metadados originais
        'content': chunk['text']  # Adiciona o conteúdo do chunk como metadado
    }

    # Adicionar ao banco vetorial
    collection.add(
        embeddings=[embedding],  # Embedding gerado
        metadatas=[metadata_with_content],  # Metadados atualizados com 'content'
        ids=[str(chunk['metadata']['id'])]  # ID único do chunk
    )

print(f"{len(embeddings)} registros inseridos na coleção '{collection_name}' com sucesso!")


12 registros inseridos na coleção 'novo' com sucesso!


Teste se os dados foram inseridos corretamente:

In [67]:
# Recuperar os dados da coleção para validação
all_data = collection.get(include=["metadatas", "embeddings"])

# Exibir os primeiros registros para validação
print(f"Total de registros na coleção: {len(all_data['metadatas'])}")
for idx, (metadata, embedding) in enumerate(zip(all_data['metadatas'], all_data['embeddings']), start=1):
    print(f"\n--- Registro {idx} ---")
    print(f"Metadados: {metadata}")
    print(f"Embedding (resumido): {embedding[:10]}...")  # Exibe apenas os primeiros 10 valores do embedding


Total de registros na coleção: 12

--- Registro 1 ---
Metadados: {'content': '<missing-text>\n\nFundamentos da Programação Web Atividade Prática\n\nATIVIDADE PRÁTICA\n\n1.  OBJETIVO\n\nAplicar os conceitos estudados na disciplina.\n\n2.  PROGRAMAS NECESSÁRIOS\n\nPara desenvolver esta atividade será necessário:\n\n ❖ HTML\n\n ❖ CSS\n\n ❖ JavaScript (livre)\n\n ❖ Acesso a um provedor de hospedagem gratuito.\n\n ❖ Um diagramador (IDE VS Code, por exemplo)\n\n3.  ORIENTAÇÕES GERAIS', 'file_name': 'ATIVIDADE_PRATICA_C1_2024.pdf', 'id': 1, 'length': 380, 'type': 'text'}
Embedding (resumido): [ 0.50097656  0.11077881  0.20092773  0.25708008 -0.37792969  0.20288086
  0.44116211 -0.39990234  0.43066406  0.18347168]...

--- Registro 2 ---
Metadados: {'content': '❖ Um diagramador (IDE VS Code, por exemplo)\n\n3.  ORIENTAÇÕES GERAIS\n\nDesenvolver um site, totalmente funcional, com no mínimo quatro links, conforme orientações, e publicá-lo num servidor de hospedagem gratuito. Procure  deixar  o  m

##Passo 6: Depurando a Consulta


Verifique se os embeddings estão disponíveis no ChromaDB.


In [14]:
results = collection.query(
    query_embeddings=[embeddings[0]],  # Use o primeiro embedding como exemplo
    n_results=2
)
print("Resultados da consulta:", results)


Resultados da consulta: {'ids': [['1', '2']], 'embeddings': None, 'documents': [[None, None]], 'uris': None, 'data': None, 'metadatas': [[{'content': '<missing-text>\n\nFundamentos da Programação Web Atividade Prática\n\nATIVIDADE PRÁTICA\n\n1.  OBJETIVO\n\nAplicar os conceitos estudados na disciplina.\n\n2.  PROGRAMAS NECESSÁRIOS\n\nPara desenvolver esta atividade será necessário:\n\n ❖ HTML\n\n ❖ CSS\n\n ❖ JavaScript (livre)\n\n ❖ Acesso a um provedor de hospedagem gratuito.\n\n ❖ Um diagramador (IDE VS Code, por exemplo)\n\n3.  ORIENTAÇÕES GERAIS', 'file_name': 'ATIVIDADE_PRATICA_C1_2024.pdf', 'id': 1, 'length': 380, 'type': 'text'}, {'content': '❖ Um diagramador (IDE VS Code, por exemplo)\n\n3.  ORIENTAÇÕES GERAIS\n\nDesenvolver um site, totalmente funcional, com no mínimo quatro links, conforme orientações, e publicá-lo num servidor de hospedagem gratuito. Procure  deixar  o  menu  visível  em  todas  as  páginas,  evite  o  uso  do VOLTAR do navegador.\n\nExemplos:\n\n<missing-te

In [15]:
# Realizar a consulta no ChromaDB
results = collection.query(
    query_embeddings=[embeddings[11]],  # Use o primeiro embedding como exemplo
    n_results=12
)

# Validar e exibir os resultados
if results and 'metadatas' in results and results['metadatas']:
    print("Resultados da consulta:")
    for idx, (metadata, score_list) in enumerate(zip(results['metadatas'], results['distances']), start=1):
        # Se score_list for uma lista, pegue o primeiro elemento
        score = score_list[0] if isinstance(score_list, list) else score_list
        print(f"\n--- Resultado {idx} ---")
        print("Metadados:", metadata)
        print(f"Similaridade: {1 - score:.4f}")  # Similaridade como 1 - distância (se for cosseno)
else:
    print("Nenhum resultado encontrado.")


Resultados da consulta:

--- Resultado 1 ---
Metadados: [{'content': '|    | Páginas   | Atendeu Totalmente  Valor  25  pontos  página                                                                                                                | cada  24  todos   | Atendeu  parcialmente  Valor de 1 a  pontos  cada página                               | Não atendeu  Valor  0  pontos  cada  página   | Nota  cada  página   |\n|---:|:----------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------|:------------------|:---------------------------------------------------------------------------------------|:----------------------------------------------|:---------------------|\n|  0 | Sobre mim | Atendeu                                                                                                                                                      | os                | Atendeu       

In [17]:
collections = client.list_collections()
print([collection.name for collection in collections])


['novo']


In [19]:
print(results['metadatas'])


[[{'content': '|    | Páginas   | Atendeu Totalmente  Valor  25  pontos  página                                                                                                                | cada  24  todos   | Atendeu  parcialmente  Valor de 1 a  pontos  cada página                               | Não atendeu  Valor  0  pontos  cada  página   | Nota  cada  página   |\n|---:|:----------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------|:------------------|:---------------------------------------------------------------------------------------|:----------------------------------------------|:---------------------|\n|  0 | Sobre mim | Atendeu                                                                                                                                                      | os                | Atendeu                                                              

In [30]:
import cohere

def search_and_generate_response(query, collection_name, cohere_api_key, n_results=2):
    """
    Realiza busca no banco vetorial e gera uma resposta usando a LLM do Cohere.

    Args:
        query (str): A pergunta ou consulta do usuário.
        collection_name (str): O nome da coleção no ChromaDB.
        cohere_api_key (str): A chave de API do Cohere.
        n_results (int): Número de resultados mais similares a buscar.

    Returns:
        str: A resposta gerada pela LLM do Cohere.
    """
    # Inicializar o cliente Cohere
    cohere_client = cohere.Client(api_key=cohere_api_key)

    # Obter a coleção no ChromaDB
    collection = client.get_collection(name=collection_name)

    # Gerar embedding para a consulta
    query_embedding = cohere_client.embed(
        model="embed-multilingual-v2.0",
        texts=[query]
    ).embeddings[0]

    # Buscar os resultados mais similares no índice vetorial
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results
    )

    # Validar se há resultados
    if not results or not results['metadatas']:
        return "Nenhum resultado relevante encontrado para a sua consulta."

    # Construir o contexto com base nos resultados encontrados
    context = "\n".join(
        f"Trecho {outer_idx + 1}.{inner_idx + 1}: {item['content']}"
        for outer_idx, metadata_list in enumerate(results['metadatas'])
        for inner_idx, item in enumerate(metadata_list)
    )

    # Criar o prompt para a LLM
    prompt = f"""
    Você é um assistente que responde perguntas, usando a arquitetura RAG. Você vai receber as pesquisas mais relevantes e precisa formular uma resposta para o usuário. Pode haver texto e tabelas nos dados fornecidos.

    {context}

    Pergunta do usuário: {query}

    Responda de forma clara e completa com base nos trechos fornecidos.
    """

    # Gerar a resposta usando a LLM do Cohere
    response = cohere_client.generate(
        model="command-r-plus-08-2024",
        prompt=prompt,
        max_tokens=300,
        temperature=0.7
    )
    return response.generations[0].text.strip()


# Teste da Função
collection_name = "novo"
query = "o que tem que fazer na atividade pratica?"
response = search_and_generate_response(query, collection_name, cohere_api_key)

print("Resposta da IA:\n", response)


Resposta da IA:
 A atividade prática visa aplicar os conceitos estudados na disciplina de Fundamentos da Programação Web, exigindo a criação de um formulário de contato visualmente atraente e funcional. Aqui está o que você precisa fazer:

1. **Criação do Formulário de Contato:**
   - Desenvolva um formulário de contato que inclua campos para nome, email e mensagem. Certifique-se de utilizar apenas HTML e CSS, sem o uso de frameworks como Bootstrap.
   - O HTML será usado para estruturar o formulário e os campos de entrada.
   - O CSS será responsável pelo estilo e layout visual do formulário.

2. **Requisitos Técnicos:**
   - **Linguagens:** Utilize apenas HTML e CSS para criar o formulário. Não é permitido o uso de frameworks como Bootstrap ou ferramentas semelhantes.
   - **Estilização:** Crie uma aparência visual atraente para o formulário usando CSS. Garanta que os estilos sejam aplicados corretamente aos elementos do formulário.
   - **Comentários no Código:** Explique seu código